In [6]:
import cv2
from ultralytics import YOLO
import numpy as np

# تحميل نموذج YOLO
model = YOLO("yolov8n.pt")  # تأكد من وجود النموذج في نفس المجلد

# تحميل الصورة
image_path = "handball.jpg"
output_path = "result_handball_yolo.jpg"
image = cv2.imread(image_path)

# كشف العناصر في الصورة
results = model(image)

ball_position = None
player_boxes = []

# تحليل النتائج
for result in results:
    for box in result.boxes:
        cls_id = int(box.cls[0])
        label = result.names[cls_id]
        x1, y1, x2, y2 = map(int, box.xyxy[0])

        if label in ["sports ball", "ball"]:
            ball_position = ((x1 + x2) // 2, (y1 + y2) // 2)
            cv2.circle(image, ball_position, 10, (0, 0, 255), -1)
        elif label in ["person"]:
            player_boxes.append((x1, y1, x2, y2))
            cv2.rectangle(image, (x1, y1), (x2, y2), (255, 255, 0), 2)

# التحقق من قرب الكرة من منطقة اليد (تقريباً أعلى يمين/يسار جسم اللاعب)
def is_possible_hand_touch(ball_position, player_boxes, threshold=50):
    if ball_position is None:
        return False, None
    for (x1, y1, x2, y2) in player_boxes:
        # فرض أن اليد على الجانبين العلويين
        left_hand = (x1 + 10, y1 + int((y2 - y1) * 0.3))
        right_hand = (x2 - 10, y1 + int((y2 - y1) * 0.3))
        for hand in [left_hand, right_hand]:
            distance = np.linalg.norm(np.array(hand) - np.array(ball_position))
            if distance < threshold:
                return True, hand
    return False, None

# التحقق من لمسة يد
handball, hand_point = is_possible_hand_touch(ball_position, player_boxes)

if handball:
    cv2.line(image, hand_point, ball_position, (0, 255, 255), 2)
    cv2.putText(image, "Possible Handball!", (30, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 0, 255), 3)

# حفظ النتيجة
cv2.imwrite(output_path, image)
print(f"تم حفظ النتيجة في: {output_path}")


0: 480x640 5 persons, 1 sports ball, 515.6ms
Speed: 66.7ms preprocess, 515.6ms inference, 41.5ms postprocess per image at shape (1, 3, 480, 640)
تم حفظ النتيجة في: result_handball_yolo.jpg


In [7]:
import cv2
import numpy as np
from ultralytics import YOLO

# تحميل YOLO
model = YOLO("yolov8n.pt")

# تحميل الصورة
image_path = "handball.jpg"
output_path = "result_hand_touch.jpg"
image = cv2.imread(image_path)

# تحليل الصورة
results = model(image)

ball_box = None
players = []

# استخراج البوكسات
for result in results:
    for box in result.boxes:
        cls_id = int(box.cls[0])
        label = result.names[cls_id]
        x1, y1, x2, y2 = map(int, box.xyxy[0])

        if label in ["sports ball", "ball"]:
            ball_box = (x1, y1, x2, y2)
            cv2.rectangle(image, (x1, y1), (x2, y2), (0, 0, 255), 2)
        elif label == "person":
            players.append((x1, y1, x2, y2))
            cv2.rectangle(image, (x1, y1), (x2, y2), (200, 200, 0), 2)

# تحديد منطقة اليد في جسم اللاعب
def get_hand_regions(player_box):
    x1, y1, x2, y2 = player_box
    width = x2 - x1
    height = y2 - y1
    # نفرض أن اليد اليسرى واليد اليمنى تقعان في أعلى جانبي الجسم
    left_hand_box = (x1, y1, x1 + width // 4, y1 + height // 2)
    right_hand_box = (x2 - width // 4, y1, x2, y1 + height // 2)
    return [left_hand_box, right_hand_box]

# التحقق من التقاطع بين البوكسات
def boxes_intersect(box1, box2):
    x1, y1, x2, y2 = box1
    a1, b1, a2, b2 = box2
    return not (x2 < a1 or x1 > a2 or y2 < b1 or y1 > b2)

# التحقق من تلامس اليد مع الكرة
handball_detected = False
for player_box in players:
    hand_boxes = get_hand_regions(player_box)
    for hand_box in hand_boxes:
        if ball_box and boxes_intersect(hand_box, ball_box):
            # رسم بوكس اليد المتلامسة
            cv2.rectangle(image, (hand_box[0], hand_box[1]), (hand_box[2], hand_box[3]), (0, 255, 255), 2)
            cv2.putText(image, "Handball!", (hand_box[0], hand_box[1]-10), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0,0,255), 2)
            handball_detected = True

# حفظ الصورة
cv2.imwrite(output_path, image)
print("تم حفظ الصورة مع التحقق من التلامس بين الكرة واليد.")


0: 480x640 5 persons, 1 sports ball, 302.1ms
Speed: 9.7ms preprocess, 302.1ms inference, 3.3ms postprocess per image at shape (1, 3, 480, 640)
تم حفظ الصورة مع التحقق من التلامس بين الكرة واليد.


In [9]:
import cv2
import numpy as np
from ultralytics import YOLO

# تحميل YOLO
model = YOLO("yolov8n.pt")

# تحميل الصورة
image_path = "handball_1.jpg"
output_path = "result_hand_touch_1.jpg"
image = cv2.imread(image_path)

# تحليل الصورة
results = model(image)

ball_box = None
players = []

# استخراج البوكسات
for result in results:
    for box in result.boxes:
        cls_id = int(box.cls[0])
        label = result.names[cls_id]
        x1, y1, x2, y2 = map(int, box.xyxy[0])

        if label in ["sports ball", "ball"]:
            ball_box = (x1, y1, x2, y2)
            ball_center = ((x1 + x2) // 2, (y1 + y2) // 2)
            cv2.rectangle(image, (x1, y1), (x2, y2), (0, 0, 255), 2)
        elif label == "person":
            players.append((x1, y1, x2, y2))
            cv2.rectangle(image, (x1, y1), (x2, y2), (200, 200, 0), 2)

# تحديد منطقة اليد الأوسع (ثلث العرض في ثلثي الطول العلوي)
def get_hand_regions(player_box):
    x1, y1, x2, y2 = player_box
    width = x2 - x1
    height = y2 - y1

    # تغطية الجانبين العلوي والوسيط للجسم
    left_hand_box = (x1, y1, x1 + width // 3, y1 + int(height * 0.7))
    right_hand_box = (x2 - width // 3, y1, x2, y1 + int(height * 0.7))
    return [left_hand_box, right_hand_box]

# التحقق من دخول مركز الكرة في منطقة اليد
def point_in_box(point, box):
    x, y = point
    x1, y1, x2, y2 = box
    return x1 <= x <= x2 and y1 <= y <= y2

# فحص التلامس
handball_detected = False
for player_box in players:
    hand_boxes = get_hand_regions(player_box)
    for hand_box in hand_boxes:
        if ball_box and point_in_box(ball_center, hand_box):
            cv2.rectangle(image, (hand_box[0], hand_box[1]), (hand_box[2], hand_box[3]), (0, 255, 255), 2)
            cv2.putText(image, "Handball!", (hand_box[0], hand_box[1]-10), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0,0,255), 2)
            handball_detected = True

# حفظ الصورة
cv2.imwrite(output_path, image)
print("تم حفظ الصورة المعدلة مع تحسين دقة التلامس.")


0: 640x640 2 persons, 1 sports ball, 332.7ms
Speed: 37.9ms preprocess, 332.7ms inference, 28.1ms postprocess per image at shape (1, 3, 640, 640)
تم حفظ الصورة المعدلة مع تحسين دقة التلامس.
